In [2]:
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import math

In [30]:
tickers = ['VTI', 'AGG', 'DBC', '^VIX']
data = yf.download(tickers, start="2011-01-01", end="2020-04-30", interval="1d")['Adj Close']
data_na = data.dropna(axis = 0)
data_na.dropna(axis=0, inplace=True)
data_na.reset_index(inplace=True)
data_na['Date'] = data_na['Date'].dt.date


[*********************100%***********************]  4 of 4 completed


In [31]:
data_na['Date'] = pd.to_datetime(data_na['Date'])


In [32]:
data_na

Ticker,Date,AGG,DBC,VTI,^VIX
0,2011-01-03,73.837639,25.471508,51.215210,17.610001
1,2011-01-04,73.858627,25.130905,51.113708,17.379999
2,2011-01-05,73.502113,25.397865,51.394768,17.020000
3,2011-01-06,73.516075,25.057262,51.277657,17.400000
4,2011-01-07,73.753784,24.965210,51.160545,17.139999
...,...,...,...,...,...
2341,2020-04-23,104.660980,10.270258,130.396225,41.380001
2342,2020-04-24,104.607498,10.137616,132.339752,35.930000
2343,2020-04-27,104.126190,9.948129,134.685135,33.290001
2344,2020-04-28,104.536201,9.957603,134.264633,33.570000


In [5]:
# Allocation initiale
initial_allocation = [0.50, 0.10, 0.20, 0.20]  # Allocations initiales pour VTI, AGG, DBC, VIX
portfolio_value = 1


In [6]:
# Créer un DataFrame pour les allocations initiales
allocation_df = pd.DataFrame(index=data_na.index, columns=[f'{col}_alloc' for col in data_na.columns[1:]])

# Initialiser les allocations du portefeuille pour chaque actif
initial_prices = data_na.iloc[0, 1:]  # Prix initiaux des actifs
allocation_df.iloc[0] = [portfolio_value * alloc / price for alloc, price in zip(initial_allocation, initial_prices)]


In [7]:
def rebalance_portfolio(row, prev_alloc, prices, initial_allocation):
    total_value = (prev_alloc * prices).sum()  # Calcul de la valeur totale du portefeuille
    new_alloc = [(total_value * alloc) / price for alloc, price in zip(initial_allocation, prices)]
    return pd.Series(new_alloc)


In [8]:
# DataFrame pour stocker les valeurs du portefeuille
portfolio_df = pd.DataFrame(index=data_na.index, columns=['Date', 'portfolio_value'])
portfolio_df['Date'] = data_na['Date']

In [9]:
# Parcourir chaque année et réallouer le portefeuille au 1er janvier de chaque année
for i in range(1, len(data_na)):
    current_year = pd.to_datetime(data_na.loc[i, 'Date']).year
    previous_year = pd.to_datetime(data_na.loc[i-1, 'Date']).year
    
    if current_year != previous_year:  # Réallocation si on change d'année
        previous_alloc = allocation_df.iloc[i-1].values.astype(float)
        current_prices = data_na.iloc[i, 1:].values.astype(float)
        
        # Réallocation basée sur les prix actuels et les allocations initiales
        allocation_df.iloc[i] = rebalance_portfolio(data_na.iloc[i],previous_alloc, current_prices, initial_allocation)
    else:
        allocation_df.iloc[i] = allocation_df.iloc[i-1]  # Conserver la même allocation
    
    # Utilisation de loc pour éviter les avertissements
    portfolio_df.loc[i, 'portfolio_value'] = (allocation_df.iloc[i] * data_na.iloc[i, 1:].values).sum()


In [10]:
# Initialisation du DataFrame des poids
weights_df = pd.DataFrame(index=data_na.index, columns=data_na.columns[1:])

# Poids fixes pour 2011
weights_2011 = initial_allocation  

# Assignation des poids pour 2011
for i in range(len(data_na)):
    current_year = pd.to_datetime(data_na.loc[i, 'Date']).year
    
    if current_year == 2011:
        weights_df.loc[i] = weights_2011  
    elif current_year > 2011:
        # Si c'est la première occurrence de l'année
        if i == 0 or pd.to_datetime(data_na.loc[i - 1, 'Date']).year != current_year:
            # Récupérer les prix de la première ligne de l'année
            first_row_prices = data_na.iloc[i, 1:].values.astype(float)
            # Calculer les poids basés sur la première ligne de l'année
            weights_df.loc[i] = fixed_weights = first_row_prices / first_row_prices.sum()
        else:
            # Pour les autres jours de la même année, conserver les mêmes poids
            weights_df.loc[i] = weights_df.loc[i - 1]



In [11]:
# Supprimer la dernière ligne du DataFrame des poids
weights_df = weights_df.drop(weights_df.index[-1]).reset_index(drop=True)


In [12]:
# Calculer les rendements quotidiens
returns_df = data_na.iloc[:, 1:].pct_change().fillna(0)
returns_df['Date'] = data_na['Date']
returns_df = returns_df[['Date'] + list(returns_df.columns[:-1])]



In [13]:
returns_df = returns_df.drop(index=returns_df.index[0]).reset_index(drop=True)

In [14]:
returns_df = returns_df.drop(columns=['Date'])

In [15]:
# Calculer le produit terme à terme
weighted_returns = returns_df.values * weights_df.values

# Somme sur les lignes pour obtenir le rendement du portefeuille
portfolio_returns = weighted_returns.sum(axis=1)

In [16]:
portfolio_returns

array([-0.004203606759099488, -0.00439414757280957, 0.002763499438648076,
       ..., 0.003776011157360968, 0.00011878580811951986,
       0.01359452227762845], dtype=object)

In [17]:
# Calcul de l'espérance des rendements (moyenne)
expected_return = portfolio_returns.mean()
expected_return = (expected_return+1)**252-1

# Calcul de la standard déviation
std_dev = portfolio_returns.std()
std_dev = std_dev * math.sqrt(252)
risk_free_rate = 0.0

# Calcul du Sharpe ratio
sharpe_ratio = (expected_return - risk_free_rate) / std_dev

# Affichage des résultats
print(f"Espérance des rendements : {expected_return:.4f}")
print(f"Standard déviation : {std_dev:.4f}")
print(f"Sharpe ratio : {sharpe_ratio:.4f}")


Espérance des rendements : 0.1334
Standard déviation : 0.1133
Sharpe ratio : 1.1769


In [18]:
# Initialisation des listes pour stocker les premières données de chaque année
weights_first_list = []
prices_first_list = []
returns_first_list = []
allocation_first_list = []

# Boucle sur les années de 2011 à 2020
for year in range(2011, 2021):
    # Filtrer les données pour l'année en question
    mask = pd.to_datetime(data_na['Date']).dt.year == year
    
    # Stocker la première ligne de chaque année dans les listes
    weights_first_list.append(weights_df.loc[mask].iloc[0])
    prices_first_list.append(data_na.loc[mask].iloc[0])
    returns_first_list.append(returns_df.loc[mask].iloc[0])
    allocation_first_list.append(allocation_df.loc[mask].iloc[0])

# Convertir les listes en DataFrames
weights_first_df = pd.DataFrame(weights_first_list).reset_index(drop=True)
prices_first_df = pd.DataFrame(prices_first_list).reset_index(drop=True)
returns_first_df = pd.DataFrame(returns_first_list).reset_index(drop=True)
allocation_first_df = pd.DataFrame(allocation_first_list).reset_index(drop=True)


## Prise en compte du cout de transaction

In [19]:
C = 0.01  

# Initialisation du vecteur des coûts de transaction
transaction_costs = np.zeros(len(weights_df))

# Calcul des coûts de transaction
for t in range(2, len(weights_df)):
    # Calcul des différences de poids
    diff_weights = weights_df.iloc[t - 1].values - weights_df.iloc[t - 2].values
    
    # Somme des valeurs absolues des différences
    sum_abs_diff = np.sum(np.abs(diff_weights))
    
    # Coût de transaction pour le temps t
    transaction_costs[t] = C * sum_abs_diff



In [20]:
portfolio_with_transac_cost = portfolio_returns +transaction_costs

In [21]:
# Calcul de l'espérance des rendements (moyenne)
expected_return = portfolio_with_transac_cost.mean()
expected_return = (expected_return+1)**252-1

# Calcul de la standard déviation
std_dev = portfolio_with_transac_cost.std()
std_dev = std_dev * math.sqrt(252)
risk_free_rate = 0.0

# Calcul du Sharpe ratio
sharpe_ratio = (expected_return - risk_free_rate) / std_dev

# Affichage des résultats
print(f"Espérance des rendements : {expected_return:.4f}")
print(f"Standard déviation : {std_dev:.4f}")
print(f"Sharpe ratio : {sharpe_ratio:.4f}")


Espérance des rendements : 0.1347
Standard déviation : 0.1133
Sharpe ratio : 1.1883


## Prise en compte de la volatilité scaling

In [22]:
# Initialisation du DataFrame pour stocker les volatilités ex-ante
volatility_df = pd.DataFrame(index=returns_df.index, columns=returns_df.columns)

# Boucle sur chaque colonne (actif) pour calculer la volatilité
for col in returns_df.columns:
    # Calcul de la variance exponentiellement pondérée sur 50 jours
    ew_var = returns_df[col].ewm(span=50, adjust=False).var()
    
    # Calcul de la volatilité ex-ante en prenant la racine carrée de la variance
    volatility_df[col] = ew_var.shift(1).apply(lambda x: x**0.5)



In [23]:
volatility_df.iloc[48:53]

Ticker,AGG,DBC,VTI,^VIX
48,0.002047,0.010439,0.008735,0.071994
49,0.002005,0.012417,0.008787,0.076133
50,0.002024,0.012272,0.009203,0.084170
51,0.001995,0.013688,0.009374,0.085929
52,0.001960,0.013469,0.009250,0.085940


In [24]:
# Définition du paramètre alpha
vol_target = 0.1  

vol_scale_df = vol_target / volatility_df


In [25]:
vol_scale_df

Ticker,AGG,DBC,VTI,^VIX
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,27.668754,5.893858,18.905163,18.480298
3,39.136400,8.331284,26.660422,5.377897
4,40.391862,9.540802,32.571576,6.568140
...,...,...,...,...
2340,10.199396,4.962047,2.627129,0.794350
2341,10.400440,5.061499,2.680198,0.809760
2342,10.605831,5.145042,2.726392,0.805257
2343,10.747953,5.196251,2.770256,0.815172
